In [2]:
#import
import pandas as pd
import numpy as np

In [3]:
#sys 모듈 설치
import sys

In [4]:
#한글폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,573 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [1]:
#한글폰트 설정
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [2]:
#필요 라이브러리 설치
!pip install beautifulsoup4
!pip install urllib3
!pip install openpyxl

#urllib : 특정 url 주소에 접근할 때 사용할 라이브러리
#beautifulsoup : url에서 html 정보를 파이썬에서 이해할 수 있는 언어로 변환하는 (parsing,파싱) 라이브러리

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#라이브러리 임포트
from bs4 import BeautifulSoup
import requests
import openpyxl

In [4]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

In [5]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + '&sort=0&photo=0&field=0&pd=5&ds=2021.05.26&de=2022.05.26&cluster_rank=16&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all' + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + '&sort=0&photo=0&field=0&pd=5&ds=2021.05.26&de=2022.05.26&cluster_rank=16&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all' + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [6]:
# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


In [7]:
#뉴스기사 내용 크롤링하는 함수 만들기(각 뉴스의 url)
def news_contents_crawler(news_url):
    contents=[]
    for i in news_url:
        #각 기사 html get하기
        news = requests.get(i)
        news_html = BeautifulSoup(news.text,"html.parser")
            #기사 내용 가져오기 (p태그의 내용 모두 가져오기) 
        contents.append(news_html.find_all('p'))
    return contents

In [8]:
#html생성해서 기사크롤링하는 함수 만들기(제목,url): 3개의 값을 반환함(제목, 링크, 내용)
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과
    articles = html.select("div.group_news > ul.list_news > li div.news_area > a")
    title = news_attrs_crawler(articles,'title')
    url = news_attrs_crawler(articles,'href')
    content = news_contents_crawler(url)
    return title, url, content #3개의 값을 반환

In [9]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
for i in url:
    title, url,content = articles_crawler(url)
    news_titles.append(title)
    news_url.append(url)
    news_contents.append(content)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(news_url)
print("\n[뉴스 내용]")
print(news_contents)

검색할 키워드를 입력해주세요:외국인 노동자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):2

크롤링할 종료 페이지:  2 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=외국인 노동자&sort=0&photo=0&field=0&pd=5&ds=2021.05.26&de=2022.05.26&cluster_rank=16&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=외국인 노동자&sort=0&photo=0&field=0&pd=5&ds=2021.05.26&de=2022.05.26&cluster_rank=16&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all&start=11']
검색된 기사 갯수: 총  20 개

[뉴스 제목]
[["휴대폰 든 상자만 훔친 택배사 외국인일용직 2명 '덜미'", '택배 물건서 최신 휴대폰만 절도..외국인노동자 검거', '방역지침 완화에 경북 시·군, 외국인 계절근로자 앞다퉈 도입', '두산건설 공사현장서 외국인 노동자 사망…중대재해법 조사', '“고용허가제, 현대판 노예제”…이주노동자 기본권·인권·노동권 침해 심각', '외국인 일손 떠난 농촌… 경범죄자·노숙자 ‘구원 등판’', '[41주년 특별기획] 외국인, 일꾼을 넘어 동반자를 꿈꾸다 ① 농업·농촌 현장에 일할 사람이 없다', '“개시장 단골은 70대 노인과 외국인 노동자뿐... 이젠 접어야죠”', '아파트 공사현장서 30대 노동자 사망…

In [10]:
###데이터 프레임으로 만들기###
import pandas as pd

#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist
    
#제목, 링크, 내용 담을 리스트 생성
news_titles_1, news_url_1, news_contents_1 = [],[],[]

#1차원 리스트로 만들기(내용 제외)
makeList(news_titles_1,news_titles)
makeList(news_url_1,news_url)
makeList(news_contents_1,news_contents)


#데이터 프레임 만들기
news_df = pd.DataFrame({'title':news_titles_1,'link':news_url_1,'content':news_contents_1})
news_df

,title,link,content
0,휴대폰 든 상자만 훔친 택배사 외국인일용직 2명 '덜미',http://www.newsis.com/view/?id=NISX20220526_00...,"[[2022.05.28 (토)], [[서울 16℃]], [[사회]], [휴대폰 든 ..."
1,택배 물건서 최신 휴대폰만 절도..외국인노동자 검거,https://ysmbc.co.kr/article/NWaMc7iR7_orUjn,[]
2,"방역지침 완화에 경북 시·군, 외국인 계절근로자 앞다퉈 도입",https://www.yna.co.kr/view/AKR2022052612310005...,"[[재난포털], [기사제보], [자동완성 기능이 켜져 있습니다.], [[송고시간],..."
3,두산건설 공사현장서 외국인 노동자 사망…중대재해법 조사,https://www.sisa-news.com/news/article.html?no...,"[[2022.05.28 (토)], [ ], [[시사뉴스 한지혜 기자] 두산건설 공사..."
4,"“고용허가제, 현대판 노예제”…이주노동자 기본권·인권·노동권 침해 심각",https://www.kgnews.co.kr/news/article.html?no=...,"[[2022.05.28 (토)], [ ], [“이주노동자들이 사는 숙소 가운데는 비..."
5,외국인 일손 떠난 농촌… 경범죄자·노숙자 ‘구원 등판’,https://www.chosun.com/national/national_gener...,[]
6,"[41주년 특별기획] 외국인, 일꾼을 넘어 동반자를 꿈꾸다 ① 농업·농촌 현장에 일...",http://www.aflnews.co.kr/news/articleView.html...,"[[[농수축산신문=박유신 기자]], [[<span style=""font-family..."
7,“개시장 단골은 70대 노인과 외국인 노동자뿐... 이젠 접어야죠”,https://www.chosun.com/national/weekend/2022/0...,[]
8,아파트 공사현장서 30대 노동자 사망… 광주 중대재해 처벌 1호 되나,https://www.nocutnews.co.kr/news/5761459,"[[\n, [\n, <img alt=""탑배너 닫기"" src=""//img.nocutn..."
9,부산 도금 공장에서 불…외국인 노동자 6명 대피,https://news.kbs.co.kr/news/view.do?ncd=546731...,"[[\n, [황현규 기자], \n, [true@kbs.co.kr], \n], [Co..."


In [12]:

news_df.to_csv('뉴스크롤링.csv', encoding = 'utf8')